In [ ]:
import pandas as pd
import numpy as np
import requests
import os
from bs4 import BeautifulSoup


### Get all Horse IDs required

In [ ]:
race_df = pd.read_csv('../preprocessing/preprocessed_races.csv')
horse_ids = list(set(race_df['horse_id']))
horse_ids.sort()
len(horse_ids)


# Scrapping Horse Data from SCMP

In [ ]:
SCMP_BASE_URL = 'https://www.scmp.com/sport/racing/ajax/stats/HorseName/'

horse_subpaths = []
horse_id_name_map = {}

for char in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    url = f'{SCMP_BASE_URL}{char}'
    resp = requests.get(url)
    if resp.status_code == 200:
        data = resp.json()['data']['result']
        print(f"Retrieved {len(data)} horses starting with {char}")
        for horse in data:
            subpath = f"{horse['horse_id']}/{horse['horse_name'].lower().replace(' ', '-')}"
            horse_subpaths.append(subpath)
            horse_id_name_map[horse['horse_id']] = horse['horse_name']

print(f"Total {len(horse_subpaths)} horses")


In [ ]:
SCMP_HORSES_BASE_URL = 'https://www.scmp.com/sport/racing/stats/horses'

SCMP_RACE_TABLE = [
    'date', 'race_number', 'track', 'distance', 'cl', 
    'rank', 'trainer', 'weight', 'jockey', 'dr', 'gr', 
    'winner', 'second', 'third', 'win_time', 'last_qtr', 
    'section_time', 'in_running', 'w/m', 'horse_weight', 
    'rt', 'odds_on', 'odds_last'
]

SCMP_HORSE_DETAILS = [
    'import_type', 'colour', 'sex', 'age', 'country', 
    'owner', 'last_win', 'health', 'bloodline_relations', 
    'sire', 'dam'
]

SCMP_RACE_TABLE_DIR = 'scmp/race_table'
SCMP_HORSE_DETAILS_DIR = 'scmp/horse_details'


In [ ]:
failed = []

def scrape_scmp_horses(subpaths):
    for subpath in subpaths:
        horse_id = subpath.split('/')[0]
        resp = requests.get(f'{SCMP_HORSES_BASE_URL}/{subpath}')

        if resp.status_code == 200:
            print(f"Processing {subpath}")
            soup = BeautifulSoup(resp.content, 'html.parser')

            race_table_element = soup.find('tbody')
            if race_table_element:
                race_table_data = []
                for row in race_table_element.find_all('tr'):
                    race_table_data.append([cell.get_text(strip=True) for cell in row.find_all('td')])

                race_table_df = pd.DataFrame(race_table_data, columns=SCMP_RACE_TABLE)
                race_table_df.to_csv(f'{SCMP_RACE_TABLE_DIR}/{horse_id}.csv', index=False)

            else:
                failed.append((subpath, SCMP_RACE_TABLE_DIR))

            horse_details_element = soup.find('div', class_='details')
            if horse_details_element:
                horse_details = []
                for detail in horse_details_element.find_all('b'):
                    if detail.contents[0] == 'Import Type / Colour / Sex / Age / Country of Origin: ':
                        for d in detail.next_sibling.split('/'):
                            horse_details.append(d.strip())
                    else:
                        horse_details.append(detail.next_sibling.strip())

                horse_details_df = pd.DataFrame([horse_details], columns=SCMP_HORSE_DETAILS)
                horse_details_df.to_csv(f'{SCMP_HORSE_DETAILS_DIR}/{horse_id}.csv', index=False)
            else:
                failed.append((subpath, SCMP_HORSE_DETAILS_DIR))


In [ ]:
# scrape_scmp_horses(horse_subpaths)


### Checking Failed Scraping Subpaths

In [ ]:
print(len(failed))
failed


### Preprocessing SCMP Horse Data

In [ ]:
scmp_horse_details_df = pd.DataFrame()

for scmp_horse in os.listdir(SCMP_HORSE_DETAILS_DIR):
    scmp_horse_id = scmp_horse.split('.')[0]
    scmp_horse_df = pd.read_csv(f'{SCMP_HORSE_DETAILS_DIR}/{scmp_horse}')

    scmp_horse_df['horse_id'] = scmp_horse_id
    scmp_horse_df['age'] = scmp_horse_df['age'].apply(lambda x: int(x.split()[0]))
    scmp_horse_details_df = pd.concat([scmp_horse_details_df, scmp_horse_df])
    print(f"Processed {horse_id}")


In [ ]:
print(scmp_horse_details_df.shape)
scmp_horse_details_df.head()


In [ ]:
SCMP_RACE_TABLE_COLS = [
    'horse_id', 'win_freq', 'place_freq', 'average_race_rank', 
    'average_horse_weight', 'average_total_weight', 'latest_horse_rating', 
    'latest_horse_class', 'average_speed', 'average_winner_speed_diff'
]

scmp_race_results_df = pd.DataFrame(columns=SCMP_RACE_TABLE_COLS)

def time_to_seconds(time_str):
    time_str = time_str.replace('.', ':')
    parts = time_str.split(':')
    return int(parts[0]) * 60 + int(parts[1])

for scmp_race in os.listdir(SCMP_RACE_TABLE_DIR):
    scmp_horse_id = scmp_race.split('.')[0]
    scmp_race_df = pd.read_csv(f'{SCMP_RACE_TABLE_DIR}/{scmp_race}')
    print(f"Processing {scmp_horse_id}")

    average_horse_weight = pd.to_numeric(scmp_race_df['weight'], errors='coerce', downcast='float').mean()
    average_total_weight = pd.to_numeric(scmp_race_df['horse_weight'], errors='coerce', downcast='float').mean()

    latest_horse_rating = scmp_race_df['rt'].iloc[0]
    latest_horse_class = scmp_race_df['cl'].iloc[0]

    scmp_race_df['section_time'] = scmp_race_df['section_time'].astype(str)
    scmp_race_df['total_time'] = scmp_race_df['section_time'].apply(lambda x: sum([float(t) for t in x.split()]))

    scmp_race_df.dropna(inplace=True)

    scmp_race_df['average_speed'] = scmp_race_df['distance'] / scmp_race_df['total_time']
    scmp_race_df['winner_speed'] = scmp_race_df['distance'] / scmp_race_df['win_time'].apply(time_to_seconds)
    scmp_race_df['winner_speed_diff'] = scmp_race_df['winner_speed'] - scmp_race_df['average_speed']
    average_speed = scmp_race_df['average_speed'].mean()
    average_winner_speed_diff = scmp_race_df['winner_speed_diff'].mean()

    total_rows = scmp_race_df['winner'].count()
    win_freq = (scmp_race_df['winner'] == horse_id_name_map[scmp_horse_id]).sum() / total_rows
    place = (scmp_race_df['winner'] == horse_id_name_map[scmp_horse_id]) | (scmp_race_df['second'] == horse_id_name_map[scmp_horse_id]) | (scmp_race_df['third'] == horse_id_name_map[scmp_horse_id])
    place_freq = place.sum() / total_rows

    average_race_rank = pd.to_numeric(scmp_race_df['rank'], errors='coerce').mean()

    row = [scmp_horse_id, win_freq, place_freq, average_race_rank, average_horse_weight, average_total_weight, latest_horse_rating, latest_horse_class, average_speed, average_winner_speed_diff]
    scmp_race_results_df = scmp_race_results_df.append(pd.Series(row, index=scmp_race_results_df.columns), ignore_index=True)


In [ ]:
print(scmp_race_results_df.shape)
scmp_race_results_df.head()


### Joining Horse Details with Aggregated Horse Race Performance data

In [ ]:
scmp_horses_df = scmp_horse_details_df.set_index('horse_id').join(scmp_race_results_df.set_index('horse_id'), how='inner').reset_index()
print(scmp_horses_df.shape)
scmp_horses_df.head()


In [ ]:
scmp_horses_df.isna().sum()


In [ ]:
scmp_horses_df.dropna(inplace=True)
scmp_horses_df.reset_index(drop=True, inplace=True)
print(scmp_horses_df.shape)
scmp_horses_df.head()


In [ ]:
scmp_missing_horse_ids = set(race_df['horse_id']) - set(scmp_horses_df['horse_id'])
print(len(scmp_missing_horse_ids))


In [ ]:
scmp_horses_df.to_csv('scmp_horses.csv', index=False)


# Scrapping Horse Data from HKJC

In [ ]:
HKJC_HORSES_BASE_URL = 'http://www.hkjc.com/english/racing/horse.asp?HorseNo='

HKJC_HORSE_PROFILE_DIR = 'hkjc/horse_profile'
HKJC_RACE_RESULTS_DIR = 'hkjc/race_results'


In [ ]:
def scrape_hkjc_horses(ids):
    horse_id = ids[i]
    resp = requests.get(f'{HKJC_HORSES_BASE_URL}{horse_id}')

    if resp.status_code == 200:
        print(f"Processing {horse_id}")
        soup = BeautifulSoup(resp.content, 'html.parser')

        horse_profile_table = soup.find('table', class_='horseProfile')
        if horse_profile_table:
            horse_profile_data = {}
            table_body = horse_profile_table.find_all('tbody')

            title = table_body[1].find('span', class_='title_text').get_text()
            horse_name = title.split(' ')[0]
            horse_profile_data['horse_name'] = horse_name

            for tbody in table_body[3:]:
                for row in tbody.find_all('tr'):
                    cells = row.find_all('td')
                    tmp = ""
                    for cell in cells:
                        tmp += cell.get_text().strip()
                    if tmp and tmp.split(":"):
                        k,v = tmp.split(":")
                        horse_profile_data[k] = v

            horse_profile_df = pd.DataFrame([horse_profile_data])
            horse_profile_df.to_csv(f'{HKJC_HORSE_PROFILE_DIR}/{horse_id}.csv', index=False)

        horse_race_results_table = soup.find('table', class_='bigborder')
        if horse_race_results_table:
            headers = [header.get_text() for header in horse_race_results_table.find_all('tr')[0].find_all('td')[:-1]]

            race_results = []
            for row in horse_race_results_table.find_all('tr')[1:]:
                cells = row.find_all('td')
                if len(cells) == 1 and "Season" in cells[0].get_text():
                    continue

                race_results.append([cell.get_text().strip() for cell in cells[:-2]])

            if race_results and race_results[0]:
                race_results_df = pd.DataFrame(race_results, columns=headers)
                race_results_df.dropna(inplace=True)
                race_results_df.to_csv(f'{HKJC_RACE_RESULTS_DIR}/{horse_id}.csv', index=False)



In [ ]:
scrape_hkjc_horses(horse_ids)


### Preprocessing HKJC Horse data

In [ ]:
HKJC_HORSE_PROFILE_COLS = [
    'horse_id', 'horse_name', 'country', 'colour', 'sex', 'import_type', 
    'total_stakes', 'last_rating', 'wins', 'places', 'total_races', 
    'win_freq', 'place_freq',
]

RACE_RESULTS_HEADERS = [
    'horse_id', 'average_rating', 'average_placing', 'average_race_class', 
    'average_speed'
]


In [ ]:
hkjc_horse_profile_df = pd.DataFrame()

for hkjc_horse in os.listdir(HKJC_HORSE_PROFILE_DIR):
    hkjc_horse_id = hkjc_horse.split('.')[0]
    hkjc_horse_df = pd.read_csv(f'{HKJC_HORSE_PROFILE_DIR}/{hkjc_horse}')
    print(f"Processed {hkjc_horse_id}")

    horse_name = hkjc_horse_df['horse_name'].iloc[0]   
    colour_sex = hkjc_horse_df['Colour / Sex'].iloc[0].split(' / ')
    colour, sex = colour_sex[0], colour_sex[-1]
    import_type = hkjc_horse_df['Import Type'].iloc[0]
    hkjc_horse_df['Total Stakes*'] = hkjc_horse_df['Total Stakes*'].replace('[\$,]', '', regex=True).astype(int)
    total_stakes = hkjc_horse_df['Total Stakes*'].iloc[0]

    last_rating = None
    if "Last Rating" in hkjc_horse_df:
        last_rating = hkjc_horse_df['Last Rating'].iloc[0]

    if "Country of Origin" in hkjc_horse_df:
        country = hkjc_horse_df['Country of Origin'].iloc[0]
    elif "Country of Origin / Age" in hkjc_horse_df:
        country = hkjc_horse_df['Country of Origin / Age'].iloc[0].split(' / ')[0]
    
    win, second, third, starts = hkjc_horse_df['No. of 1-2-3-Starts*'].iloc[0].split('-')
    win = int(win)
    place = win + int(second) + int(third)
    win_freq, place_freq = 0, 0
    if starts != '0':
        win_freq = win / int(starts)
        place_freq = int(place) / int(starts)

    row = [hkjc_horse_id, horse_name, country, colour, sex, import_type, total_stakes, last_rating, win, place, starts, win_freq, place_freq]
    hkjc_horse_profile_df = hkjc_horse_profile_df.append(pd.Series(row, index=HKJC_HORSE_PROFILE_COLS), ignore_index=True)


In [ ]:
print(hkjc_horse_profile_df.shape)
hkjc_horse_profile_df.head()


In [ ]:
hkjc_race_results_df = pd.DataFrame()

def time_to_seconds(time_str):
    parts = time_str.split('.')
    return int(parts[0]) * 60 + int(parts[1])

for hkjc_race in os.listdir(HKJC_RACE_RESULTS_DIR):
    hkjc_horse_id = hkjc_race.split('.')[0]
    hkjc_race_df = pd.read_csv(f'{HKJC_RACE_RESULTS_DIR}/{hkjc_race}')
    print(f"Processed {hkjc_horse_id}")

    hkjc_race_df = hkjc_race_df[(hkjc_race_df != '--').all(axis=1)]
    hkjc_race_df.dropna(inplace=True)
    hkjc_race_df.reset_index(drop=True, inplace=True)

    average_rating = pd.to_numeric(hkjc_race_df['Rtg.'], errors='coerce', downcast='float').mean()
    average_placing = pd.to_numeric(hkjc_race_df['Pla.'], errors='coerce', downcast='float').mean()
    average_race_class = pd.to_numeric(hkjc_race_df['RaceClass'], errors='coerce', downcast='float').mean()

    hkjc_race_df['Finish Time'] = hkjc_race_df['Finish Time'].apply(time_to_seconds)
    hkjc_race_df['Average Speed'] = hkjc_race_df['Dist.'].astype(int) / hkjc_race_df['Finish Time']
    average_race_speed = hkjc_race_df['Average Speed'].mean()

    row = [hkjc_horse_id, average_rating, average_placing, average_race_class, average_race_speed]
    hkjc_race_results_df = hkjc_race_results_df.append(pd.Series(row, index=RACE_RESULTS_HEADERS), ignore_index=True)


In [ ]:
print(hkjc_race_results_df.shape)
hkjc_race_results_df.head()


In [ ]:
hkjc_horses_df = hkjc_horse_profile_df.set_index('horse_id').join(hkjc_race_results_df.set_index('horse_id'), how='inner').reset_index()
print(hkjc_horses_df.shape)
hkjc_horses_df.head(10)


In [ ]:
hkjc_horses_df.isna().sum()


In [ ]:
hkjc_horses_df.dropna(inplace=True)
hkjc_horses_df.reset_index(drop=True, inplace=True)
hkjc_horses_df.shape


In [ ]:
hkjc_missing_horse_ids = set(race_df['horse_id']) - set(hkjc_horses_df['horse_id'])
print(len(hkjc_missing_horse_ids))


In [ ]:
hkjc_horses_df.to_csv('hkjc_horses.csv', index=False)
